## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Ocean Park\r312 Ocean Dr BH5\rMiami Beach,2/27/2023,746,"$3,200,000.00",2180,"$1,869.16",Eloy Carmenate,The Corcoran Group,Selim Ozutez,LoKation,Ocean Park 312 Ocean Dr BH5 Miami Beach,Ocean Park,312 Ocean Dr BH5 Miami Beach,"(25.7723523, -80.1327671)",25.772352,-80.132767
2,Bayside Village East\r2122 Fisher Island Dr 21...,3/1/2023,196,"$3,050,000.00",1847,"$1,651.33",Karla Abaunza,Luxury Living Realty,Jonathan Corso,Coldwell Banker Realty,Bayside Village East 2122 Fisher Island Dr 212...,Bayside Village East,2122 Fisher Island Dr 2122 Miami Beach,"(25.7584111, -80.1416702)",25.758411,-80.141670
3,Acqualina\r17875 Collins Ave 2406\rSunny Isles...,2/28/2023,104,"$3,000,000.00",2736,"$1,096.49",Viviane Wolak,One Sotheby's Int'l Realty,Steve Silverman,The Keyes Company,Acqualina 17875 Collins Ave 2406 Sunny Isles B...,Acqualina,17875 Collins Ave 2406 Sunny Isles Beach,"(25.9408209, -80.1209181)",25.940821,-80.120918
4,The Gables Club\r60 Edgewater Dr 5E\rCoral Gables,2/27/2023,27,"$2,500,000.00",3100,$806.45,Toni Schrager,Brown Harris Stevens,Al Arostegui,One Sotheby's Int'l Realty,The Gables Club 60 Edgewater Dr 5E Coral Gables,The Gables Club,60 Edgewater Dr 5E Coral Gables,"(25.706287, -80.2532669)",25.706287,-80.253267
5,Portofino Tower\r300 S Pointe Dr 1404\rMiami B...,2/28/2023,30,"$2,200,000.00",2040,"$1,078.43",John Lennon,South Pointe Drive Realty Inc,Jeanne Mockridge,South Pointe Drive Realty Inc,Portofino Tower 300 S Pointe Dr 1404 Miami Beach,Portofino Tower,300 S Pointe Dr 1404 Miami Beach,"(25.7677055, -80.1348768)",25.767706,-80.134877
6,Brickell Key One Condo\r520 Brickell Key Dr A3...,2/28/2023,125,"$1,800,000.00",2456,$732.90,Nicole Exposito,One Sotheby's Int'l Realty,Stephanie Chin-Quee,Cohen & Company,Brickell Key One Condo 520 Brickell Key Dr A31...,Brickell Key One Condo,520 Brickell Key Dr A313 Miami,"(25.767054, -80.18486209999999)",25.767054,-80.184862
7,South Pointe Towers Condo\r400 S Pointe Dr 160...,3/2/2023,83,"$1,720,000.00",1279,"$1,344.80",Solange Santos,"One Way Realty Group, LLC",Delphine Jullien,Condoideas Realty Group LLC,South Pointe Towers Condo 400 S Pointe Dr 1609...,South Pointe Towers Condo,400 S Pointe Dr 1609 Miami Beach,"(25.7672947, -80.13418370000001)",25.767295,-80.134184
8,St Tropez\r150 Sunny Isles Blvd 1-UPH2\rSunny ...,2/28/2023,110,"$1,579,000.00",2911,$542.43,Deborah Revah,Sunny International Realty,Deborah Revah,Sunny International Realty,St Tropez 150 Sunny Isles Blvd 1-UPH2 Sunny Is...,St Tropez,150 Sunny Isles Blvd 1-UPH2 Sunny Isles Beach,"(25.9294501, -80.12361829999999)",25.929450,-80.123618
9,Four Seasons Residences\r1425 Brickell Ave 57A...,2/27/2023,264,"$1,571,000.00",1579,$994.93,Brigitte Nachtigall,Great Properties International LLC,Daniel Stone,"D Stone Realty, LLC.",Four Seasons Residences 1425 Brickell Ave 57A ...,Four Seasons Residences,1425 Brickell Ave 57A Miami,"(25.759088, -80.19181669999999)",25.759088,-80.191817
10,0Sunset Harbour South\r1800 Sunset Harbour Dr ...,2/27/2023,1,"$1,490,000.00",1220,"$1,221.31",Daniel Sepe,Calibre International Realty,Paul Sasseville,"Compass Florida, LLC.",0Sunset Harbour South 1800 Sunset Harbour Dr 2...,,0Sunset Harbour South 1800 Sunset Harbour Dr 2...,"(25.7944878, -80.14480089999999)",25.794488,-80.144801


In [9]:
df.at[1,'Agent']=('Eloy Carmenate & Mick Duchon')

df.at[10,'building_name']=('Sunset Harbour')
df.at[10,'address_only']=('1800 South Sunset Harbour Drive 2108')

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [12]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

In [13]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

## HTML Popup Formatter

In [14]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [15]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Listing Broker']
    buyers_agent = row['Buyer Agent']
    psf = row['Sq. Ft.']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [16]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February 26th - March 4th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [17]:
m.save('index.html')

## Data snagger

In [18]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [19]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Ocean Park , 312 Ocean Dr BH5 Miami Beach | Price $3,200,000 | $1,869 psf | Listing agent: Eloy Carmenate & Mick Duchon with The Corcoran Group | Buyer's agent: Selim Ozutez with LoKation | Days on market: 746
Least Expensive
Sunset Harbour, 1800 South Sunset Harbour Drive 2108 | Price $1,490,000 | $1,221 psf | Listing agent: Daniel Sepe with Calibre International Realty | Buyer's agent: Paul Sasseville with Compass Florida, LLC. | Days on market: 1
Highest Price Per Square Foot
Ocean Park , 312 Ocean Dr BH5 Miami Beach | Price $3,200,000 | $1,869 psf | Listing agent: Eloy Carmenate & Mick Duchon with The Corcoran Group | Buyer's agent: Selim Ozutez with LoKation | Days on market: 746
Lowest Price Per Square Foot
St Tropez , 150 Sunny Isles Blvd 1-UPH2 Sunny Isles Beach | Price $1,579,000 | $542 psf | Listing agent: Deborah Revah with Sunny International Realty | Buyer's agent: Deborah Revah with Sunny International Realty | Days on market: 110
Most Days on Market
Ocean 

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [20]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [21]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_03062023
